In [2]:
import numpy as np
from numpy import expand_dims
import keras.backend as K
from keras.layers import Input, Lambda
from keras.models import Model
from keras.models import load_model
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

from yolo3.utils import get_random_data
from yolo3.model import preprocess_true_boxes, yolo_body, yolo_loss
from yolo3.yolo import YOLO, detect_video

from timeit import default_timer as timer


from PIL import Image, ImageFont, ImageDraw

Using TensorFlow backend.


In [3]:
# set up logging tensorboard
log_dir = "Logs/"
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [4]:
# load classes
def get_classes(classes_path):
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

def get_anchors(anchors_path):
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)

def get_annotations(annotation_path):
    with open(annotation_path) as f:
        lines = f.readlines()
    return lines

annotation_path = 'annotations.txt'
classes_path = 'classes.txt'      
anchors_path = 'yolo/keras-yolo3/keras-yolo3/model_data/yolo_anchors.txt' # using default yolo anchors

classes = get_classes(classes_path)
num_classes = len(classes)
anchors = get_anchors(anchors_path)
annotations = get_annotations(annotation_path)

# separate a random part of dataset for validation
np.random.seed(10101)
np.random.shuffle(annotations)
np.random.seed(None)
val_split = 0.2
num_val = int(len(annotations)*val_split)
num_train = len(annotations) - num_val

print("-------------------CLASS NAMES-------------------")
print(classes)
print("-------------------CLASS NAMES-------------------")

print(annotations[3])

print(annotations[:num_train])



-------------------CLASS NAMES-------------------
['Airplane', 'Helicopter', 'Quadcopter', 'UAV']
-------------------CLASS NAMES-------------------
F:\Projects\AVObjectTracking\dataset\OIDv4_ToolKit-master\OID\Dataset\train\UAV\85.jpg 27,79,397,261,3

['F:\\Projects\\AVObjectTracking\\dataset\\OIDv4_ToolKit-master\\OID\\Dataset\\train\\Helicopter\\0be2d0e4f6bae641.jpg 142,50,924,636,1\n', 'F:\\Projects\\AVObjectTracking\\dataset\\OIDv4_ToolKit-master\\OID\\Dataset\\train\\UAV\\1078096850_0_691_2508_2048_1000x541_80_0_0_fc109e98d6ff18e4ebd80be6ac3562a9.jpg 26,85,1000,391,3\n', 'F:\\Projects\\AVObjectTracking\\dataset\\OIDv4_ToolKit-master\\OID\\Dataset\\train\\Helicopter\\26ecae23c364218f.jpg 0,74,812,767,1 332,41,1023,528,1 721,218,1023,477,1\n', 'F:\\Projects\\AVObjectTracking\\dataset\\OIDv4_ToolKit-master\\OID\\Dataset\\train\\UAV\\85.jpg 27,79,397,261,3\n', 'F:\\Projects\\AVObjectTracking\\dataset\\OIDv4_ToolKit-master\\OID\\Dataset\\train\\UAV\\images_(2).jpg 20,1,205,107,3\n', 'F

In [5]:
# Data augmentation
def data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes):
    '''data generator for fit_generator'''
    n = len(annotation_lines)
    i = 0
    while True:
        image_data = []
        box_data = []
        for b in range(batch_size):
            if i==0:
                np.random.shuffle(annotation_lines)
            image, box = get_random_data(annotation_lines[i], input_shape, random=True)
            image_data.append(image)
            box_data.append(box)
            i = (i+1) % n
        image_data = np.array(image_data)
        box_data = np.array(box_data)
        y_true = preprocess_true_boxes(box_data, input_shape, anchors, num_classes)
        yield [image_data, *y_true], np.zeros(batch_size)
        
def data_generator_wrapper(annotation_lines, batch_size, input_shape, anchors, num_classes):
    n = len(annotation_lines)
    if n==0 or batch_size<=0: return None
    return data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes)

In [6]:
# Create model
def create_model(input_shape, anchors, num_classes, freeze_body, weights_path, load_pretrained=True):
    K.clear_session() # get a new session
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)
    
    print(input_shape)
    print(num_anchors)
    print(num_classes)

    size_dict = {0:32, 1:16, 2:8}
    h_scaled = h//size_dict[0]
    w_scaled = w//size_dict[0]
    anchors_divided = num_anchors//3
    output_classes = num_classes+5
    y_true = [Input(shape=(h//size_dict[l], w//size_dict[l], anchors_divided, output_classes)) for l in range(3)]

    model_body = yolo_body(image_input, num_anchors//3, num_classes)
    print('Create YOLOv3 model with {} anchors and {} classes.'.format(num_anchors, num_classes))

    if load_pretrained:
        model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
        print('Load weights {}.'.format(weights_path))
        if freeze_body in [1, 2]:
            # Freeze darknet53 body or freeze all but 3 output layers.
            num = (185, len(model_body.layers)-3)[freeze_body-1]
            for i in range(num): model_body.layers[i].trainable = False
            print('Freeze the first {} layers of total {} layers.'.format(num, len(model_body.layers)))

    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': 0.5})(
        [*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)

    return model

In [18]:
# Here we do the training
input_shape = (416,416)


# Stage 1: Create a stock yolo model to start training with frozen layers
model = create_model(input_shape, anchors, num_classes, freeze_body=2, weights_path='yolo/keras-yolo3/keras-yolo3/model_data/yolo.h5')

# Stage 2: Continue training with all layers
# model = create_model(input_shape, anchors, num_classes, freeze_body=2, weights_path='Logs/trained_weights_stage_1.h5')

model.summary()

model.compile(optimizer=Adam(lr=1e-3), loss={'yolo_loss': lambda y_true, y_pred: y_pred})

batch_size = 32
print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
model.fit_generator(data_generator_wrapper(annotations[:num_train], batch_size, input_shape, anchors, num_classes),
    steps_per_epoch=max(1, num_train//batch_size),
    validation_data=data_generator_wrapper(annotations[num_train:], batch_size, input_shape, anchors, num_classes),
    validation_steps=max(1, num_val//batch_size),
    epochs=500,
    initial_epoch=0
)
model.save_weights(log_dir + 'trained_weights_stage_1.h5')

print("Stage 1 complete")
    

for i in range(len(model.layers)):
    model.layers[i].trainable = True
model.compile(optimizer=Adam(lr=1e-4), loss={'yolo_loss': lambda y_true, y_pred: y_pred}) # recompile to apply the change
print('Unfreeze all of the layers.')

batch_size = 4
print('Train on {} samples, val on {} samples, with batch size {}.'.format(num_train, num_val, batch_size))
model.fit_generator(data_generator_wrapper(annotations[:num_train], batch_size, input_shape, anchors, num_classes),
    steps_per_epoch=max(1, num_train//batch_size),
    validation_data=data_generator_wrapper(annotations[num_train:], batch_size, input_shape, anchors, num_classes),
    validation_steps=max(1, num_val//batch_size),
    epochs=100,
    initial_epoch=50
)

model.save_weights(log_dir + 'trained_weights_final_mk2.h5')
model.save("trained_models/mk2.h5")

print("Stage 2 complete")

(416, 416)
9
4
Create YOLOv3 model with 9 anchors and 4 classes.


f:\venv3\lib\site-packages\keras\engine\saving.py:1140: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((1, 1, 1024, 27) vs (255, 1024, 1, 1)).
  weight_values[i].shape))
f:\venv3\lib\site-packages\keras\engine\saving.py:1140: UserWarning: Skipping loading of weights for layer conv2d_59 due to mismatch in shape ((27,) vs (255,)).
  weight_values[i].shape))
f:\venv3\lib\site-packages\keras\engine\saving.py:1140: UserWarning: Skipping loading of weights for layer conv2d_67 due to mismatch in shape ((1, 1, 512, 27) vs (255, 512, 1, 1)).
  weight_values[i].shape))
f:\venv3\lib\site-packages\keras\engine\saving.py:1140: UserWarning: Skipping loading of weights for layer conv2d_67 due to mismatch in shape ((27,) vs (255,)).
  weight_values[i].shape))
f:\venv3\lib\site-packages\keras\engine\saving.py:1140: UserWarning: Skipping loading of weights for layer conv2d_75 due to mismatch in shape ((1, 1, 256, 27) vs (255, 256, 1, 1)).
  weight_values[i].shape)

Load weights yolo/keras-yolo3/keras-yolo3/model_data/yolo.h5.
Freeze the first 249 layers of total 252 layers.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, None, None, 3 0           batch_normalization_1[0][0]     

Epoch 1/500
13/13 [==============================] - 46s 4s/step - loss: 5310.1236 - val_loss: 2339.3049
Epoch 2/500
13/13 [==============================] - 39s 3s/step - loss: 1270.8619 - val_loss: 695.7341
Epoch 3/500
13/13 [==============================] - 42s 3s/step - loss: 480.0764 - val_loss: 370.5096
Epoch 4/500
13/13 [==============================] - 38s 3s/step - loss: 283.0605 - val_loss: 243.9205
Epoch 5/500
13/13 [==============================] - 36s 3s/step - loss: 209.6134 - val_loss: 197.0359
Epoch 6/500
13/13 [==============================] - 36s 3s/step - loss: 169.9304 - val_loss: 169.2361
Epoch 7/500
13/13 [==============================] - 36s 3s/step - loss: 146.5999 - val_loss: 134.3675
Epoch 8/500
13/13 [==============================] - 36s 3s/step - loss: 127.9518 - val_loss: 120.5801
Epoch 9/500
13/13 [==============================] - 36s 3s/step - loss: 113.4658 - val_loss: 104.7510
Epoch 10/500
13/13 [==============================] - 38s 3s/step - lo

13/13 [==============================] - 39s 3s/step - loss: 21.8462 - val_loss: 21.7283
Epoch 82/500
13/13 [==============================] - 38s 3s/step - loss: 21.8108 - val_loss: 22.2283
Epoch 83/500
13/13 [==============================] - 39s 3s/step - loss: 22.2980 - val_loss: 21.3636
Epoch 84/500
13/13 [==============================] - 38s 3s/step - loss: 21.4181 - val_loss: 22.9423
Epoch 85/500
13/13 [==============================] - 39s 3s/step - loss: 22.2373 - val_loss: 21.1601
Epoch 86/500
13/13 [==============================] - 39s 3s/step - loss: 21.1360 - val_loss: 22.4852
Epoch 87/500
13/13 [==============================] - 39s 3s/step - loss: 21.8522 - val_loss: 21.8892
Epoch 88/500
13/13 [==============================] - 39s 3s/step - loss: 21.1484 - val_loss: 20.9472
Epoch 89/500
13/13 [==============================] - 38s 3s/step - loss: 21.6924 - val_loss: 21.9905
Epoch 90/500
13/13 [==============================] - 38s 3s/step - loss: 20.3976 - val_loss: 2

Epoch 161/500
13/13 [==============================] - 40s 3s/step - loss: 18.2500 - val_loss: 19.4523
Epoch 162/500
13/13 [==============================] - 39s 3s/step - loss: 18.4594 - val_loss: 19.4109
Epoch 163/500
13/13 [==============================] - 38s 3s/step - loss: 17.9011 - val_loss: 18.4538
Epoch 164/500
13/13 [==============================] - 38s 3s/step - loss: 18.0399 - val_loss: 18.4352
Epoch 165/500
13/13 [==============================] - 40s 3s/step - loss: 18.3468 - val_loss: 18.8026
Epoch 166/500
13/13 [==============================] - 40s 3s/step - loss: 18.2503 - val_loss: 18.9055
Epoch 167/500
13/13 [==============================] - 42s 3s/step - loss: 18.2076 - val_loss: 18.2981
Epoch 168/500
13/13 [==============================] - 40s 3s/step - loss: 17.9937 - val_loss: 18.7444
Epoch 169/500
13/13 [==============================] - 38s 3s/step - loss: 18.0645 - val_loss: 18.4565
Epoch 170/500
13/13 [==============================] - 38s 3s/step - loss

13/13 [==============================] - 40s 3s/step - loss: 16.7825 - val_loss: 16.9117
Epoch 241/500
13/13 [==============================] - 39s 3s/step - loss: 17.3840 - val_loss: 18.4279
Epoch 242/500
13/13 [==============================] - 39s 3s/step - loss: 16.8039 - val_loss: 17.5786
Epoch 243/500
13/13 [==============================] - 39s 3s/step - loss: 16.9027 - val_loss: 17.4575
Epoch 244/500
13/13 [==============================] - 38s 3s/step - loss: 17.3253 - val_loss: 17.8274
Epoch 245/500
13/13 [==============================] - 41s 3s/step - loss: 17.3053 - val_loss: 16.7745
Epoch 246/500
13/13 [==============================] - 40s 3s/step - loss: 17.2465 - val_loss: 16.9423
Epoch 247/500
13/13 [==============================] - 40s 3s/step - loss: 16.3256 - val_loss: 18.2350
Epoch 248/500
13/13 [==============================] - 40s 3s/step - loss: 16.6748 - val_loss: 17.1989
Epoch 249/500
13/13 [==============================] - 39s 3s/step - loss: 17.2471 - va

13/13 [==============================] - 40s 3s/step - loss: 16.7125 - val_loss: 18.2809
Epoch 320/500
13/13 [==============================] - 40s 3s/step - loss: 16.2641 - val_loss: 16.7321
Epoch 321/500
13/13 [==============================] - 39s 3s/step - loss: 16.5735 - val_loss: 16.9198
Epoch 322/500
13/13 [==============================] - 41s 3s/step - loss: 15.7754 - val_loss: 16.3733
Epoch 323/500
13/13 [==============================] - 39s 3s/step - loss: 16.6379 - val_loss: 17.2491
Epoch 324/500
13/13 [==============================] - 38s 3s/step - loss: 16.4276 - val_loss: 17.3078
Epoch 325/500
13/13 [==============================] - 38s 3s/step - loss: 16.5403 - val_loss: 16.2553
Epoch 326/500
13/13 [==============================] - 39s 3s/step - loss: 16.3716 - val_loss: 17.0654
Epoch 327/500
13/13 [==============================] - 39s 3s/step - loss: 16.7460 - val_loss: 19.2491
Epoch 328/500
13/13 [==============================] - 40s 3s/step - loss: 15.9846 - va

13/13 [==============================] - 36s 3s/step - loss: 16.2934 - val_loss: 17.0977
Epoch 399/500
13/13 [==============================] - 36s 3s/step - loss: 16.3153 - val_loss: 16.6302
Epoch 400/500
13/13 [==============================] - 36s 3s/step - loss: 15.7777 - val_loss: 16.7730
Epoch 401/500
13/13 [==============================] - 36s 3s/step - loss: 16.2208 - val_loss: 16.2356
Epoch 402/500
13/13 [==============================] - 36s 3s/step - loss: 16.2284 - val_loss: 16.4593
Epoch 403/500
13/13 [==============================] - 38s 3s/step - loss: 16.1668 - val_loss: 16.6099
Epoch 404/500
13/13 [==============================] - 39s 3s/step - loss: 16.1024 - val_loss: 15.8118
Epoch 405/500
13/13 [==============================] - 38s 3s/step - loss: 15.9295 - val_loss: 15.9008
Epoch 406/500
13/13 [==============================] - 38s 3s/step - loss: 16.2019 - val_loss: 17.2916
Epoch 407/500
13/13 [==============================] - 38s 3s/step - loss: 16.4898 - va

13/13 [==============================] - 40s 3s/step - loss: 15.4896 - val_loss: 16.6495
Epoch 478/500
13/13 [==============================] - 40s 3s/step - loss: 15.6132 - val_loss: 16.4624
Epoch 479/500
13/13 [==============================] - 39s 3s/step - loss: 15.9541 - val_loss: 16.8916
Epoch 480/500
13/13 [==============================] - 41s 3s/step - loss: 15.7703 - val_loss: 15.9246
Epoch 481/500
13/13 [==============================] - 39s 3s/step - loss: 15.6707 - val_loss: 16.1957
Epoch 482/500
13/13 [==============================] - 39s 3s/step - loss: 15.5139 - val_loss: 16.8540
Epoch 483/500
13/13 [==============================] - 40s 3s/step - loss: 15.8293 - val_loss: 16.8340
Epoch 484/500
13/13 [==============================] - 40s 3s/step - loss: 15.8615 - val_loss: 17.7808
Epoch 485/500
13/13 [==============================] - 37s 3s/step - loss: 15.9547 - val_loss: 16.3788
Epoch 486/500
13/13 [==============================] - 37s 3s/step - loss: 15.8705 - va

In [18]:
# Functions for predicting image based on yolo.py functions
def preprocess_image(image):
    boxed_image = letterbox_image(image, tuple(reversed((416,416))))
    image_data = np.array(boxed_image, dtype='float32')

    print(image_data.shape)
    image_data /= 255.
    image_data = np.expand_dims(image_data, 0)  # Add batch dimension.

    return image_data

def letterbox_image(image, size):
    '''resize image with unchanged aspect ratio using padding'''
    iw, ih = image.size
    w, h = size
    scale = min(w/iw, h/ih)
    nw = int(iw*scale)
    nh = int(ih*scale)

    image = image.resize((nw,nh), Image.BICUBIC)
    new_image = Image.new('RGB', size, (128,128,128))
    new_image.paste(image, ((w-nw)//2, (h-nh)//2))
    return new_image

def predict_img(model2, image_data):
    # model2 = YOLO(**params)
    out_boxes, out_scores, out_classes = model2.sess.run(
                [model2.boxes, model2.scores, model2.classes],
                feed_dict={
                    model2.yolo_model.input: image_data,
                    model2.input_image_shape: [416,416],
                    K.learning_phase(): 0
                })
    return out_boxes, out_scores, out_classes

def draw_predictions(predictions, path):
    out_boxes = predictions[0]
    out_scores = predictions[1]
    out_classes = predictions[2]
    
    # We had some trouble with the fonts and colors, and this ain't a great solution but it works
    colors = [(255,0,0),(0,255,0),(0,0,255),(100,124,53),(255,255,0),(40,200,0),(166,166,166),(255,0,255),(123,212,2),(255,255,255),(4,4,4),(4,4,4),(4,4,4),(4,4,4),(4,4,4),(4,4,4),(4,4,4),(4,4,4),(4,4,4),(4,4,4),(4,4,4),(4,4,4),(4,4,4),(4,4,4),(4,4,4),(4,4,4),(4,4,4)]
    
    font = ImageFont.truetype(font='font/FiraMono-Medium.otf',
                    size=np.floor(3e-2 * 416 + 0.5).astype('int32'))
    thickness = 3
    class_names = get_classes('classes.txt')
    image = Image.open(path)
    og_w, og_h = image.size
    ratio_w = 416/og_w
    ratio_h = 416/og_h
    
    for i, c in reversed(list(enumerate(out_classes))):
            predicted_class = class_names[c]
            box = out_boxes[i]
            score = out_scores[i]
            
            if score <= 0.5:
                continue
            
            label = '{} {:.2f}'.format(predicted_class, score)
            
            draw = ImageDraw.Draw(image)
            label_size = draw.textsize(label, font)

            top, left, bottom, right = box
            
            # rescale the bounding boxes for original image size
            top = top/ratio_h
            left = left/ratio_w
            bottom = bottom/ratio_h
            right = right/ratio_w
            
            top = max(0, np.floor(top + 0.5).astype('int32'))
            left = max(0, np.floor(left + 0.5).astype('int32'))
            bottom = min(og_h, np.floor(bottom + 0.5).astype('int32'))
            right = min(og_w, np.floor(right + 0.5).astype('int32'))
            print(label, (left, top), (right, bottom))

            if top - label_size[1] >= 0:
                text_origin = np.array([left, top - label_size[1]])
            else:
                text_origin = np.array([left, top + 1])

            for i in range(thickness):
                draw.rectangle(
                    [left + i, top + i, right - i, bottom - i],
                    outline=colors[c])
            draw.rectangle(
                [tuple(text_origin), tuple(text_origin + label_size)],
                fill=colors[c])
            draw.text(text_origin, label, fill=(0, 0, 0), font=font)
            del draw
    print("saving image")
    image.save('./Saved_images/test3.jpg')
    print("saved")



In [11]:
# Create model for predictions
params = {
        "model_path": './Logs/trained_weights_final_mk2.h5',
        "anchors_path": './yolo/keras-yolo3/keras-yolo3/model_data/yolo_anchors.txt',
        "classes_path": 'classes.txt'
}

model2 = YOLO(**params)

./Logs/trained_weights_final_mk2.h5 model, anchors, and classes loaded.


In [13]:
# detect_video function based on yolo.py
def detect_video(model2, video_path, output_path="./Saved_videos/"):
    # model2 = YOLO(**params)
    import cv2
    vid = cv2.VideoCapture(video_path)
    if not vid.isOpened():
        raise IOError("Couldn't open webcam or video")
    video_FourCC    = int(vid.get(cv2.CAP_PROP_FOURCC))
    video_fps       = vid.get(cv2.CAP_PROP_FPS)
    video_size      = (int(vid.get(cv2.CAP_PROP_FRAME_WIDTH)),
                        int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    isOutput = True if output_path != "" else False
    if isOutput:
        print("!!! TYPE:", type(output_path), type(video_FourCC), type(video_fps), type(video_size))
        out = cv2.VideoWriter(output_path, video_FourCC, video_fps, video_size)
    accum_time = 0
    curr_fps = 0
    fps = "FPS: ??"
    prev_time = timer()
    while True:
        return_value, frame = vid.read()
        if frame is None:
            break
        image = Image.fromarray(frame)
        image = model2.detect_image(image)
        result = np.asarray(image)
        curr_time = timer()
        exec_time = curr_time - prev_time
        prev_time = curr_time
        accum_time = accum_time + exec_time
        curr_fps = curr_fps + 1
        if accum_time > 1:
            accum_time = accum_time - 1
            fps = "FPS: " + str(curr_fps)
            curr_fps = 0
        cv2.putText(result, text=fps, org=(3, 15), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=0.50, color=(255, 0, 0), thickness=2)
        cv2.namedWindow("result", cv2.WINDOW_NORMAL)
        cv2.imshow("result", result)
        if isOutput:
            out.write(result)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()

In [19]:
# Predict Image
path = 'testing/testpic.jpg'
image = Image.open(path)
image = preprocess_image(image)

#model2 = YOLO(**params)


predictions = predict_img(model2, image)

print(predictions)

draw_predictions(predictions, path)


(416, 416, 3)
(array([[124.37332 ,  34.514847, 288.0768  , 375.61224 ],
       [119.94514 ,  11.722199, 286.05844 , 363.4407  ]], dtype=float32), array([0.8739855, 0.42559  ], dtype=float32), array([0, 3]))
Airplane 0.87 (85, 204) (925, 474)
saving image
saved


In [14]:
# Predict video, displays the video during process, but currently crashes when the video ends
video_path = 'testing/testvideo2.mp4'
detect_video(model2, video_path, './Saved_videos/t2.mp4')

!!! TYPE: <class 'str'> <class 'int'> <class 'float'> <class 'tuple'>
(416, 416, 3)
Found 2 boxes for img
Helicopter 0.53 (63, 57) (640, 360)
Airplane 0.75 (5, 83) (632, 360)
1.5343588910000108
(416, 416, 3)
Found 2 boxes for img
Helicopter 0.58 (75, 60) (639, 360)
Airplane 0.73 (7, 87) (627, 360)
0.08519205800075724
(416, 416, 3)
Found 3 boxes for img
UAV 0.33 (2, 55) (636, 360)
Helicopter 0.59 (74, 60) (640, 360)
Airplane 0.72 (6, 86) (628, 360)
0.0801965590007967
(416, 416, 3)
Found 3 boxes for img
UAV 0.34 (0, 56) (637, 360)
Helicopter 0.56 (71, 60) (640, 360)
Airplane 0.74 (5, 87) (629, 360)
0.07832010399943101
(416, 416, 3)
Found 2 boxes for img
Helicopter 0.52 (70, 59) (640, 360)
Airplane 0.72 (7, 88) (628, 360)
0.07099255400044058
(416, 416, 3)
Found 2 boxes for img
Helicopter 0.65 (74, 64) (640, 360)
Airplane 0.74 (12, 91) (622, 360)
0.07224537399997644
(416, 416, 3)
Found 2 boxes for img
Helicopter 0.62 (74, 66) (640, 360)
Airplane 0.75 (11, 94) (623, 360)
0.06877301300028194

Found 1 boxes for img
Airplane 0.62 (0, 0) (640, 360)
0.06716950900045049
(416, 416, 3)
Found 1 boxes for img
Airplane 0.64 (0, 0) (640, 360)
0.06525155800045468
(416, 416, 3)
Found 1 boxes for img
Airplane 0.64 (0, 0) (640, 360)
0.06509374999950523
(416, 416, 3)
Found 1 boxes for img
Airplane 0.65 (0, 0) (640, 360)
0.06452301100034674
(416, 416, 3)
Found 1 boxes for img
Airplane 0.65 (0, 0) (640, 360)
0.06875080400004663
(416, 416, 3)
Found 1 boxes for img
Airplane 0.65 (0, 0) (640, 360)
0.06624720900072134
(416, 416, 3)
Found 1 boxes for img
Airplane 0.65 (0, 0) (640, 360)
0.06628812300004938
(416, 416, 3)
Found 1 boxes for img
Airplane 0.67 (0, 0) (640, 360)
0.06500052599949413
(416, 416, 3)
Found 1 boxes for img
Airplane 0.66 (0, 0) (640, 360)
0.06719318100022065
(416, 416, 3)
Found 1 boxes for img
Airplane 0.67 (0, 0) (640, 360)
0.067896011000812
(416, 416, 3)
Found 1 boxes for img
Airplane 0.66 (0, 0) (640, 360)
0.06642459700015024
(416, 416, 3)
Found 1 boxes for img
Airplane 0.6

Found 1 boxes for img
Airplane 0.33 (0, 0) (640, 306)
0.06710229399959644
(416, 416, 3)
Found 1 boxes for img
Airplane 0.42 (0, 0) (640, 360)
0.06647866100047395
(416, 416, 3)
Found 1 boxes for img
Airplane 0.40 (0, 0) (640, 360)
0.06481144899953506
(416, 416, 3)
Found 1 boxes for img
Airplane 0.36 (0, 0) (640, 360)
0.06609670699981507
(416, 416, 3)
Found 1 boxes for img
Airplane 0.38 (0, 0) (640, 360)
0.06506014199931087
(416, 416, 3)
Found 1 boxes for img
Airplane 0.38 (0, 0) (640, 360)
0.06651489799969568
(416, 416, 3)
Found 1 boxes for img
Airplane 0.39 (0, 0) (640, 360)
0.0682139639993693
(416, 416, 3)
Found 1 boxes for img
Airplane 0.39 (0, 0) (640, 360)
0.0661463870001171
(416, 416, 3)
Found 1 boxes for img
Airplane 0.39 (0, 0) (640, 360)
0.06700731799992354
(416, 416, 3)
Found 1 boxes for img
Airplane 0.39 (0, 0) (640, 360)
0.06897319600011542
(416, 416, 3)
Found 1 boxes for img
Airplane 0.39 (0, 0) (640, 360)
0.06647807599983935
(416, 416, 3)
Found 1 boxes for img
Airplane 0.3

Found 0 boxes for img
0.0629338259996075
(416, 416, 3)
Found 0 boxes for img
0.06280173499999364
(416, 416, 3)
Found 0 boxes for img
0.06234116899941
(416, 416, 3)
Found 0 boxes for img
0.06245981699976255
(416, 416, 3)
Found 0 boxes for img
0.06302675699953397
(416, 416, 3)
Found 0 boxes for img
0.06264801799989073
(416, 416, 3)
Found 0 boxes for img
0.062976492999951
(416, 416, 3)
Found 0 boxes for img
0.0632950310000524
(416, 416, 3)
Found 0 boxes for img
0.061936420999700204
(416, 416, 3)
Found 0 boxes for img
0.062017077999371395
(416, 416, 3)
Found 0 boxes for img
0.06464984200010804
(416, 416, 3)
Found 0 boxes for img
0.06253258400010964
(416, 416, 3)
Found 0 boxes for img
0.06248962500012567
(416, 416, 3)
Found 0 boxes for img
0.0629332409998824
(416, 416, 3)
Found 0 boxes for img
0.06362262900074711
(416, 416, 3)
Found 0 boxes for img
0.06359252800029935
(416, 416, 3)
Found 0 boxes for img
0.06473196000024473
(416, 416, 3)
Found 1 boxes for img
Quadcopter 0.37 (181, 66) (384, 

0.06522438000047259
(416, 416, 3)
Found 1 boxes for img
Quadcopter 0.39 (163, 71) (375, 163)
0.0695661449999534
(416, 416, 3)
Found 1 boxes for img
Quadcopter 0.43 (165, 70) (377, 163)
0.06557272599911812
(416, 416, 3)
Found 0 boxes for img
0.062461862999953155
(416, 416, 3)
Found 0 boxes for img
0.06287420999979076
(416, 416, 3)
Found 0 boxes for img
0.06188118800037046
(416, 416, 3)
Found 1 boxes for img
Quadcopter 0.36 (158, 72) (379, 162)
0.06575449699994351
(416, 416, 3)
Found 1 boxes for img
Quadcopter 0.39 (158, 71) (379, 162)
0.06626298999981373
(416, 416, 3)
Found 0 boxes for img
0.0644549199996618
(416, 416, 3)
Found 0 boxes for img
0.06517177699970489
(416, 416, 3)
Found 0 boxes for img
0.06304166100017028
(416, 416, 3)
Found 1 boxes for img
Quadcopter 0.35 (154, 69) (384, 161)
0.06599267100045836
(416, 416, 3)
Found 0 boxes for img
0.06294843800060335
(416, 416, 3)
Found 0 boxes for img
0.06285375300012674
(416, 416, 3)
Found 0 boxes for img
0.0631983000002947
(416, 416, 3)

...

Found 1 boxes for img
Quadcopter 0.90 (247, 176) (303, 196)
0.06666919899998902
(416, 416, 3)
Found 1 boxes for img
Quadcopter 0.90 (246, 176) (304, 195)
0.06666598399988288
(416, 416, 3)
Found 1 boxes for img
Quadcopter 0.87 (247, 176) (304, 195)
0.0660081590003756
(416, 416, 3)
Found 1 boxes for img
Quadcopter 0.70 (259, 177) (294, 195)
0.0675415259993315
(416, 416, 3)
Found 0 boxes for img
0.0635615510000207
(416, 416, 3)
Found 1 boxes for img
Quadcopter 0.86 (290, 163) (354, 189)
0.06676096099999995
(416, 416, 3)
Found 1 boxes for img
Quadcopter 0.79 (290, 164) (351, 190)
0.06854623700019147
(416, 416, 3)
Found 1 boxes for img
Quadcopter 0.81 (279, 165) (363, 188)
0.06904420899991237
(416, 416, 3)
Found 1 boxes for img
Quadcopter 0.68 (285, 165) (356, 188)
0.06689100699986739
(416, 416, 3)
Found 1 boxes for img
Quadcopter 0.68 (285, 165) (356, 188)
0.06881626500035054
(416, 416, 3)
Found 1 boxes for img
Quadcopter 0.94 (286, 162) (359, 187)
0.06798368200088589
(416, 416, 3)
Found 1

Found 0 boxes for img
0.06440523899982509
(416, 416, 3)
Found 1 boxes for img
Quadcopter 0.39 (301, 173) (332, 184)
0.07168544800060772
(416, 416, 3)
Found 1 boxes for img
Quadcopter 0.52 (302, 173) (331, 184)
0.06776128899946343
(416, 416, 3)
Found 1 boxes for img
Quadcopter 0.52 (302, 174) (331, 184)
0.06799917100033781
(416, 416, 3)
Found 0 boxes for img
0.0625884010005393
(416, 416, 3)
Found 0 boxes for img
0.06300980699961656
(416, 416, 3)
Found 1 boxes for img
Quadcopter 0.41 (303, 172) (333, 183)
0.06615340099961031
(416, 416, 3)
Found 1 boxes for img
Quadcopter 0.33 (303, 172) (333, 184)
0.06715256000006775
(416, 416, 3)
Found 1 boxes for img
Quadcopter 0.52 (302, 172) (334, 184)
0.06786883200038574
(416, 416, 3)
Found 1 boxes for img
Quadcopter 0.41 (303, 172) (334, 184)
0.06581703600022593
(416, 416, 3)
Found 1 boxes for img
Quadcopter 0.39 (303, 172) (334, 184)
0.06619109900020703
(416, 416, 3)
Found 1 boxes for img
Quadcopter 0.47 (308, 171) (335, 184)
0.06659555599981104
(

Found 1 boxes for img
Helicopter 0.95 (66, 34) (442, 309)
0.06661075200008781
(416, 416, 3)
Found 1 boxes for img
Helicopter 0.94 (67, 34) (441, 306)
0.06709761899946898
(416, 416, 3)
Found 1 boxes for img
Helicopter 0.93 (68, 36) (440, 304)
0.06823295899994264
(416, 416, 3)
Found 1 boxes for img
Helicopter 0.93 (69, 35) (440, 307)
0.06765783699938765
(416, 416, 3)
Found 1 boxes for img
Helicopter 0.93 (69, 35) (440, 308)
0.06567120999989129
(416, 416, 3)
Found 2 boxes for img
UAV 0.35 (4, 38) (452, 298)
Helicopter 0.96 (64, 39) (444, 312)
0.0678597730002366
(416, 416, 3)
Found 2 boxes for img
UAV 0.31 (5, 38) (450, 300)
Helicopter 0.95 (66, 39) (442, 314)
0.07004687500011642
(416, 416, 3)
Found 2 boxes for img
UAV 0.50 (7, 41) (451, 301)
Helicopter 0.95 (65, 42) (443, 314)
0.06806463100019755
(416, 416, 3)
Found 2 boxes for img
UAV 0.52 (9, 41) (448, 302)
Helicopter 0.94 (67, 42) (442, 314)
0.06822945299973071
(416, 416, 3)
Found 1 boxes for img
Helicopter 0.96 (70, 45) (439, 314)
0.0

Found 2 boxes for img
UAV 0.67 (5, 74) (459, 348)
Helicopter 0.86 (53, 62) (460, 329)
0.06819058499968378
(416, 416, 3)
Found 2 boxes for img
UAV 0.67 (6, 74) (457, 348)
Helicopter 0.85 (53, 62) (461, 329)
0.06837878599981195
(416, 416, 3)
Found 2 boxes for img
UAV 0.67 (1, 64) (460, 325)
Helicopter 0.85 (52, 62) (461, 329)
0.06747957300012786
(416, 416, 3)
Found 2 boxes for img
UAV 0.63 (7, 74) (456, 349)
Helicopter 0.85 (52, 61) (461, 329)
0.07108548599990172
(416, 416, 3)
Found 2 boxes for img
UAV 0.62 (8, 74) (456, 349)
Helicopter 0.84 (52, 62) (461, 329)
0.06780483199963783
(416, 416, 3)
Found 2 boxes for img
UAV 0.64 (8, 75) (456, 349)
Helicopter 0.82 (52, 62) (461, 329)
0.0688536710003973
(416, 416, 3)
Found 2 boxes for img
UAV 0.66 (1, 63) (460, 326)
Helicopter 0.79 (56, 76) (461, 355)
0.06905209999968065
(416, 416, 3)
Found 2 boxes for img
UAV 0.68 (1, 63) (459, 326)
Helicopter 0.79 (56, 75) (461, 356)
0.06769144400004734
(416, 416, 3)
Found 2 boxes for img
UAV 0.70 (1, 62) (4

Helicopter 0.93 (55, 42) (471, 303)
0.06657451399951242
(416, 416, 3)
Found 1 boxes for img
Airplane 0.99 (0, 11) (549, 353)
0.06876278500021726
(416, 416, 3)
Found 1 boxes for img
Airplane 0.98 (0, 11) (550, 353)
0.06519603299966548
(416, 416, 3)
Found 1 boxes for img
Airplane 0.99 (0, 11) (550, 352)
0.06565630600016448
(416, 416, 3)
Found 1 boxes for img
Airplane 0.99 (0, 11) (549, 352)
0.06693747299959796
(416, 416, 3)
Found 1 boxes for img
Airplane 0.99 (0, 11) (549, 353)
0.06734660500023892
(416, 416, 3)
Found 1 boxes for img
Airplane 0.99 (0, 9) (549, 353)
0.06652571099948545
(416, 416, 3)
Found 1 boxes for img
Airplane 0.98 (0, 11) (550, 354)
0.0657331639995391
(416, 416, 3)
Found 1 boxes for img
Airplane 0.98 (0, 9) (550, 356)
0.06741206500009866
(416, 416, 3)
Found 1 boxes for img
Airplane 0.98 (0, 10) (550, 355)
0.0655569460004699
(416, 416, 3)
Found 1 boxes for img
Airplane 0.99 (0, 10) (549, 355)
0.06650613199963118
(416, 416, 3)
Found 1 boxes for img
Airplane 0.98 (0, 10) 